In [1]:
import os
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

In [2]:
train = pd.read_json('../data/train.json/train.json')
train.head()

,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."


In [3]:
vectorizer = CountVectorizer(max_features = 2000)
ingredients = train['ingredients']
words_list = [' '.join(x) for x in ingredients]

#Make label encoder
le = preprocessing.LabelEncoder()
le.fit(train["cuisine"])

print le.transform(train["cuisine"])[0:100] 

[ 6 16  4  7  7 10 17  9 13  9  9  3  9 13  9  7  1  9 18 19 18 13 16  3  9
  3  2  9  3 13  9  2 13 18  9  2  9  4 16 16  9  0 13  7 13  3  5 16 16 16
 11 16  9 16  9 10 11  7  9  8 18 18  7 10  9 18 12  5  5 16 17  7 14  9  9
 14 14 19 11 13  2 16  5  7  7  9  9  7 12 17  9 16 16  6 13 13 16  7  9  9]


In [4]:
#create a bag of words and convert to a array and then print the shape
bag_of_words = vectorizer.fit(words_list)
bag_of_words = vectorizer.transform(words_list).toarray()
print(bag_of_words.shape)

vectorizer = TfidfVectorizer(min_df=1)
tfidf = vectorizer.fit_transform(words_list).toarray()
print tfidf.shape

(39774, 2000)
(39774, 3010)


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [5]:
#Initilize a random forest classifier with 500 trees and fit it with the bag of words we created 
forest = RandomForestClassifier(n_estimators = 500) 
forest = forest.fit( bag_of_words, train["cuisine"])

incorrect = 0
for i in xrange(bag_of_words.shape[0]):
    p = forest.predict(bag_of_words[i,:])
    a = train["cuisine"][i]
    if(not(a==p[0])):
        incorrect += 1
        
print float(incorrect)/bag_of_words.shape[0]

forest2 = RandomForestClassifier(n_estimators = 500) 
forest2 = forest.fit( tfidf, train["cuisine"])

incorrect = 0
for i in xrange(bag_of_words.shape[0]):
    p = forest2.predict(bag_of_words[i,:])
    a = train["cuisine"][i]
    if(not(a==p[0])):
        incorrect += 1

0.00035198873636


ValueError: Number of features of the model must  match the input. Model n_features is 3010 and  input n_features is 2000 

In [6]:
incorrect = 0
for i in xrange(bag_of_words.shape[0]):
    p = forest2.predict(tfidf[i,:])
    a = train["cuisine"][i]
    if(not(a==p[0])):
        incorrect += 1
        
print float(incorrect)/bag_of_words.shape[0]

0.00035198873636


# Neural Network Testing

In [8]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

In [ ]:
batch_size = 128
nb_epoch = 20

X_train = bag_of_words
y_train = le.transform(train["cuisine"])

nb_classes = len(le.classes_)

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()
model.add(Dense(512, input_shape=(784,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

rms = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=rms)

model.fit(X_train, Y_train,
          batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=2,
          validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test,
                       show_accuracy=True, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
print train_acc

In [ ]:
#Now read the test json file in 
test = pd.read_json('../data/test.json/test.json')
test.head()

In [ ]:
#Do the same thing we did with the training set and create a array using the count vectorizer. 
test_ingredients = test['ingredients']
test_ingredients_words = [' '.join(x) for x in test_ingredients]
test_ingredients_array = vectorizer.transform(test_ingredients_words).toarray()

In [ ]:
# Use the random forest to make cusine predictions
result = forest.predict(test_ingredients_array)

In [ ]:
print type(result)
print result.shape
print result[0]

for i in xrange(len(result)):
    print result[i], test

In [ ]:
# Copy the results to a pandas dataframe with an "id" column and
# a "cusine" column
output = pd.DataFrame( data={"id":test["id"], "cuisine":result} )

# Use pandas to write the comma-separated output file
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )